In [32]:
import torch.nn as nn
import torch.nn.functional as F
import torch
import numpy as np
import torch.optim as optim
import os
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
from PIL import Image
import h5py
n_epochs = 1
from torch.utils.data import DataLoader
path = 'A2_dataset/dataset/'

In [33]:
def preprocess(path, resize_shape, center_crop_shape, to_crop = True):
    data = []
    count = 64
    for f in os.listdir(path):
        img = Image.open(path + f)
        size = img.size
        # print(size)
        img = np.array(img)
        img.resize(size[1], size[0], 3)
        # print(img.shape)
        img = img.transpose(2, 0, 1)
        if(to_crop):
            img = img[:, center_crop_shape[0]:center_crop_shape[1], center_crop_shape[2]:center_crop_shape[3]]
        img = img.flatten()
        img.resize(resize_shape)
        img = torch.Tensor(img)
        data.append(img)
        count -= 1
        # if(count == 0):
        #     break

    return data
    
with h5py.File('A2_dataset/train_dataset.h5', 'w') as hdf5_file:
    train_images = preprocess(path + 'images/train/', (3, 128, 128), (29, 221, 64, 256))
    hdf5_file.create_dataset('images', shape = (len(train_images),3, 128, 128), data = train_images)
    train_masks = preprocess(path + 'masks/train/', (3, 128, 128), (29, 221, 64, 256))
    hdf5_file.create_dataset('masks', shape = (len(train_masks), 3, 128, 128), data = train_masks)

with h5py.File('A2_dataset/val_dataset.h5', 'w') as hdf5_file:
    val_images = preprocess(path + 'images/val/', (3, 128, 128), (0, 128, 0, 128), False)
    hdf5_file.create_dataset('images', shape = (len(val_images), 3, 128, 128), data = val_images)
    val_masks = preprocess(path + 'masks/val/', (3, 128, 128), (0, 128, 0, 128), False)
    hdf5_file.create_dataset('masks', shape = (len(val_masks), 3, 128, 128), data = val_masks)

